<a href="https://colab.research.google.com/github/SarahPendhari/CardioCare-heart-llm-qna/blob/main/CardioLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install datasets -q

In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!wget -c https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-11-29.tar.gz -O cord19.tar.gz

--2025-04-10 16:23:00--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-11-29.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.92.186.178, 52.92.133.234, 3.5.77.152, ...
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.92.186.178|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [18]:
!mkdir cord19
!tar -xvzf cord19.tar.gz -C cord19

mkdir: cannot create directory ‘cord19’: File exists
2020-11-29/changelog
2020-11-29/cord_19_embeddings.tar.gz
2020-11-29/document_parses.tar.gz
2020-11-29/metadata.csv


In [23]:
!tar -xzf /content/cord19/2020-11-29/document_parses.tar.gz -C /content/cord19/2020-11-29/

In [24]:
import os

fulltext_dir = "/content/cord19/2020-11-29/document_parses/pdf_json"
print(os.listdir(fulltext_dir)[:5])  # should now show some JSON files

['407dfb371c1bb8508e94ada9bea7cf86e55399b3.json', '7208e9f85f74ece1f748843b09781fce0c7be9bd.json', 'cb24e4462716b5703697753dfdc9bfcaecb6452d.json', '0a15308c6bdc2ab87432c5d9aa541675dcab8c04.json', '1e2eff380a2a67997c1c2cfb38c7a4eddc396f09.json']


In [25]:
import json

# Load one JSON fulltext file
file_path = "/content/cord19/2020-11-29/document_parses/pdf_json/407dfb371c1bb8508e94ada9bea7cf86e55399b3.json"
with open(file_path, 'r') as f:
    paper = json.load(f)

# Extract full body text
body_text = paper.get("body_text", [])
full_text = " ".join([section["text"] for section in body_text])

print(full_text[:1000])  # print the first 1000 characters

"B lack swan" events are by definition unexpected (even if the "black swan" has been visible before) and come with catastrophic consequences. One such occurrence is the current coronavirus pandemic, which is likely to have a considerable impact on the practice of medicine in as yet unknown ways. A likely consequence will be a significant economic downturn and constrained budgets, resulting in even higher pressure to optimize our diagnostic and therapeutic pathways. To survive the cost pressure, diagnostic studies will have to evolve significantly: they must be fast, use the most cost effective approach, and result in a therapeutic consequences or management guidance. Shorter examinations require focus on the core question, standardized image acquisition and procedures, and rapid, increasingly automized data analysis and reporting. Cost-effectiveness requires more research, but also using the knowledge we have to reduce unnecessary diagnostic and therapeutic procedures. In this issue of

In [26]:
import os
import json

# Folder where your fulltext .json files are
fulltext_dir = "/content/cord19/2020-11-29/document_parses/pdf_json"
files = [f for f in os.listdir(fulltext_dir) if f.endswith(".json")]

# Let's collect chunks from the first N papers (you can increase later)
chunk_list = []
max_files = 100  # Start small for now

for fname in files[:max_files]:
    fpath = os.path.join(fulltext_dir, fname)
    with open(fpath, 'r') as f:
        paper = json.load(f)
        paper_id = paper.get("paper_id", "")
        title = paper.get("metadata", {}).get("title", "")

        for section in paper.get("body_text", []):
            text = section.get("text", "").strip()
            if len(text) > 100:  # Avoid very short lines
                chunk_list.append({
                    "paper_id": paper_id,
                    "title": title,
                    "text": text
                })

print(f"Extracted {len(chunk_list)} chunks.")

Extracted 3766 chunks.


In [27]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
import numpy as np

# Load a lightweight embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Get just the text chunks
texts = [chunk["text"] for chunk in chunk_list]

# Generate embeddings (batch processing is handled by the model)
embeddings = model.encode(texts, show_progress_bar=True)

# Convert to a numpy array for easier handling later
embeddings = np.array(embeddings)
print("Embeddings shape:", embeddings.shape)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.7 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/118 [00:00<?, ?it/s]

Embeddings shape: (3766, 384)


In [28]:
import pickle

# Save both chunks and embeddings
with open("cord19_chunks.pkl", "wb") as f:
    pickle.dump(chunk_list, f)

with open("cord19_embeddings.npy", "wb") as f:
    np.save(f, embeddings)

In [29]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 71.7 MB/s eta 0:00:00


In [30]:
import faiss

# Dimensionality of embeddings
embedding_dim = embeddings.shape[1]

# Create FAISS index (Flat L2 distance)
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the index
index.add(embeddings)

# Check how many vectors are indexed
print(f"Number of vectors in index: {index.ntotal}")

Number of vectors in index: 3766


In [31]:
def search(query, top_k=5):
    # Encode the query
    query_embedding = model.encode([query])

    # Search the index
    distances, indices = index.search(query_embedding, top_k)

    # Print results
    print("\n🔎 Search Results:\n")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {chunk_list[idx]['text'][:300]}...\n")

In [32]:
search("What are the symptoms of COVID-19?")


🔎 Search Results:

1. The most common symptoms of COVID-19 in children were a fever and cough. Other symptoms included fatigue, myalgia, nausea, vomiting, and diarrhea, which seems to be milder than adults with COVID-19 (Table 1) (20, 25, 81, 82) . Within 2,135 pediatric patients <18-years old who reported with COVID-19,...

2. We describe a cohort of 90 COVID-19 patients who were hospitalized in the Central Hospital of Wuhan. According to the COVID-19 Guidelines (seventh version), 20 out of 90 patients were included in the severe group, accounting for approximately 22.2% of the total cases, similar to the situations in pr...

3. Previous studies on the clinical characteristics of COVID-19 have shown that patients with COVID-19 may have mild or severe symptoms of acute respiratory infection; patients with mild disease have symptoms such as fever, dry cough, fatigue and abnormal chest CT, but their prognosis is good. In contr...

4. Among the 912 cases with COVID-19 we retrieved, 90 pa